# PRACTICA 6: QUICK START with REDIS SINGLE NODE

Primero importamos la librería Redis.

In [1]:
import redis

Creamos una conexión al servidor de Redis.

In [9]:
r = redis.Redis(host='127.0.0.1', port=16379)

Reiniciamos la BBDD de Redis para eliminar los datasets existentes.

In [10]:
r.flushall()

True

A modo de prueba, guardamos un par de claves/valores en la base de datos. 
Para el caso de  _Strings_:

In [11]:
# Guardamos el valor "funciona" con la clave "El plan"
r.set('El plan', 'funciona')
# Consultamos el valor asignado a la clave "El plan" en la base de datos
print(f"El plan: {r.get('El plan')}")

El plan: b'funciona'


In [12]:
# Guardamos el valor "funciona" con la clave "El plan"
r.set('El plan', 'funciona')
# Consultamos el valor asignado a la clave "El plan" en la base de datos
print(f"El plan: {r.get('El plan')}")

El plan: b'funciona'


Para el resto de tipos:

In [13]:
# Caso: int
r.set("Uno",1)
print(f"Uno es: {r.get('Uno')}")
# Caso: float
r.set("Uno",1.0)
print(f"Uno es: {r.get('Uno')}")
# Caso: bool
r.set("Verdadero",bytes(True)) # Conversión a int/float/bytes primero
print(f"Verdadero es: {bool(r.get('Verdadero'))}")

Uno es: b'1'
Uno es: b'1.0'
Verdadero es: True


Sobrescribimos el valor de la clave usada anteriormente:

In [15]:
# Guardamos con la misma "key" un nuevo valor
r.set("El plan","no funciona")
# Consultamos el valor asignado a la clave "El plan" en la base de datos
print(f"El plan: {r.get('El plan')}")

El plan: b'no funciona'


### PIPELINES: 
* Los pipelines son una subclase de la Redis _class_ que ofrece la posibilidad de almacenar mediante _buffering_ múltiples comandos lanzados contra el servidor en una misma petición.
* Se usan para mejorar significativamente el rendimiento de grupos de comandos, al reducir el número de comandos de back-and-forth de paquetes TCP intercambiados entre cliente y servidor.

In [16]:
# Creamos un pipeline de la base de datos Redis
pipe = r.pipeline()
# Guardamos las claves y valores en la BBDD
pipe.set('Chamartin', "28005")
pipe.set('Salamanca', "28004")
pipe.set('Retiro', "28007")
# Consultamos los valores de las claves
print(f"Chamartin: {r.get('Chamartin')}")
print(f"Salamanca: {r.get('Salamanca')}")
print(f"Retiro: {r.get('Retiro')}")
print(f"Centro: {r.get('Centro')}") # No hay valor asignado a esta clave 
pipe.execute()

Chamartin: b'28005'
Salamanca: b'28004'
Retiro: b'28007'
Centro: None


[True, True, True]

Mismo caso de antes, solo que concatenando peticiones en el mismo pipeline.

In [17]:
# Creamos un pipeline de la base de datos Redis
pipe = r.pipeline()
# Guardamos las claves y valores en la BBDD
pipe.set('Chamartin', "28005").set('Salamanca', "28004").set('Retiro', "28007")\
    .get('Chamartin').get('Salamanca').get('Retiro').get('Centro').execute()

[True, True, True, b'28005', b'28004', b'28007', None]

### PUBLISH/SUBSCRIPTION
* Redis incluye un sistema de objetos _PubSub_ que permite la subscripción a canales para escuchar nuevos mensajes publicados. 

In [1]:
import redis 
redis = redis.Redis(host='127.0.0.1', port=16379)
consumer = redis.pubsub()
consumer.subscribe('channel-football', 'channel-basket')

Recogen las publicaciones de los distintos canales. Como no hay publicaiones, no se recibe nada.

In [2]:
consumer.get_message()

{'type': 'subscribe',
 'pattern': None,
 'channel': b'channel-football',
 'data': 1}

Se muestran los canales disponibles

In [3]:
consumer.channels

{b'channel-football': None, b'channel-basket': None}

Publicamos distintos partidos en el canal de futbol:

In [19]:
redis.publish("channel-football","Real Madrid - Celta")
redis.publish("channel-football","Atletico - Alaves")
redis.publish("channel-football","Granada - Farsa")

1

{'type': 'message', 'pattern': None, 'channel': b'channel-football', 'data': b'Real Madrid - Celta'}
{'type': 'message', 'pattern': None, 'channel': b'channel-football', 'data': b'Atletico - Alaves'}


Publicamos otros tantos partidos de basket:

In [20]:
redis.publish("channel-basket","Lakers - Pelicans")
redis.publish("channel-basket","Suns - Grizzlies")
redis.publish("channel-basket","Jazz - Mavs")

1

{'type': 'message', 'pattern': None, 'channel': b'channel-football', 'data': b'Granada - Farsa'}
{'type': 'message', 'pattern': None, 'channel': b'channel-basket', 'data': b'Lakers - Pelicans'}
{'type': 'message', 'pattern': None, 'channel': b'channel-basket', 'data': b'Suns - Grizzlies'}
{'type': 'message', 'pattern': None, 'channel': b'channel-basket', 'data': b'Jazz - Mavs'}


Se recibe el mensaje recién publicado. De uno en uno.

In [27]:
consumer.get_message()

RuntimeError: pubsub connection not set: did you forget to call subscribe() or psubscribe()?

Cerramos el consumidor

In [24]:
consumer.close()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\egeah\miniconda3\envs\SSDD\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\egeah\miniconda3\envs\SSDD\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\egeah\AppData\Local\Temp\ipykernel_21168\4166509208.py", line 6, in reading_by_time
  File "C:\Users\egeah\miniconda3\envs\SSDD\lib\site-packages\redis\client.py", line 3617, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "C:\Users\egeah\miniconda3\envs\SSDD\lib\site-packages\redis\client.py", line 3497, in parse_response
    raise RuntimeError(
RuntimeError: pubsub connection not set: did you forget to call subscribe() or psubscribe()?
Exception in thread Thread-4:
Traceback (most recent call last):
  File "C:\Users\egeah\miniconda3\envs\SSDD\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\egeah\minicon